## Curso de Extensão - Python para PLN - USP

Primeiro devemos carregar os códigos do spaCy!

In [ ]:
import spacy

In [ ]:
!pip install --upgrade spacy

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.0.5)


In [ ]:
!python -m spacy download pt_core_news_lg

2021-03-23 19:00:23.718891: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 578.1MB 19kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


1. Dada uma palavra, encontrar no córpus as 3 outras palavras que mais são próximas semanticamente e as 3 palavras que são mais distantes.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
infile = open('/content/drive/My Drive/arquivos_txt/corpus_teste.txt', 'r')
conteudo = infile.read()
conteudo

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'Giants batem os Patriots no Super Bowl XLII\nAzarões acabam com a invencibilidade de New England e ficam com o título da temporada\n04/02/2008 - 01h07m - Atualizado em 04/02/2008 - 09h49m\n\nCom um passe de Eli Manning para Plaxico Burress a 39 segundos do fim, o New York Giants anotou o touchdown decisivo e derrubou o favorito New England Patriots por 17 a 14 neste domingo, em Glendale, no Super Bowl XLII. O resultado, uma das maiores zebras da história do Super Bowl, acabou com a temporada perfeita de Tom Brady e companhia, que esperavam fazer história ao levantar o troféu da NFL sem sofrer uma derrota no ano. \n\nA vitória dos Giants, porém, também ficará para a história. Pela primeira vez, irmãos quarterbacks triunfam no Super Bowl em temporadas consecutivas. No ano passado, Peyton Manning, irmão de Eli, chegou ao título máximo da NFL pelo Indianapolis Colts.\n\nA partida\n\nOs Giants começaram com a posse de bola, e mostraram logo que iriam alongar ao máximo suas posses de bola. 

In [ ]:
# Vamos assumir que a palavra que eu quero encontrar similaridade com as demais seja "título"
# Primeiro preciso saber qual a posição da palavra que eu quero comparar com todas as outras na lista de tokens:

nlp = spacy.load("pt_core_news_lg")
doc = nlp(conteudo)

# DICA! Sabemos que temos várias palavras REPETIDAS no nosso corpus. Para evitar esse problema, usamos a função set(lista). Como o resultado do set de um outro tipo, também temos qu e fazer o cast.
# Fica assim: list(set(lista)). Certo??
tokens = [token for token in doc]

index_palavra = 0

# Para eu saber qual a posição do token na lista, basta eu usar a função index!
for token in tokens:
  # Verifico qual o token que é o que eu quero comparar. O comando 'break' para o comando de repetição assim que encontrar o token 'título' 
  if token.orth_ == 'título':
    index_palavra = tokens.index(token)
    break

# Certo, sabemos onde está a palavra que queremos comparar com as outras na nossa lista de tokens.
# Vou criar minha lista de similaridade. Será nela que vou pegar as informações de proximidade, com os resultados da análise!
lista_similaridade = []

for token in tokens:

  # Aqui eu calculo a similaridade do token analisado com o meu token que quero analisar. Por exemplo, na primeira rodada, será analisada as palavras Giants e título; Depois batem e títulos, e por aí vai...
  similaridade = token.similarity(tokens[index_palavra])
  # Nossa variável info vai receber uma tupla contendo o valor da similaridade entre as palavras e qual palavra eu analisei junto a palavra 'título'. 
  info = (similaridade, token.orth_)
  # Por fim, salvarei nossa tupla na lista.
  if info not in lista_similaridade:
    lista_similaridade.append(info)

# Com a lista em mãos, podemos analisar quais são as palavras mais próximas semanticamente e as mais distantes.
# Primeiramente iremos ORDENAR nossa lista. Em ordem, saberemos quais as 3 primeiras e as 3 últimas.
# Para ordenar, basta usar a função .sort() na nossa lista, lembrem! 
# Quando se usa essa função, a lista será ordenada na ordem que o primeiro elemento da tupla está selecionado. Assim, a ordem será pelo valor de similaridade!
lista_similaridade.sort()

# Vamos pegar as 3 palavras mais próximas de 'título'. Como não colocamos nenhuma condição, a própria palavra título foi comparada com ela mesmo. Nosso slice então vai pegar 1 a mais, olha só:
print("Lista das palavras semanticamente MAIS PRÓXIMAS de \'título\': ", lista_similaridade[-4:]) 

# Agora as 3 palavras mais distantes de 'título'. Mais uma vez usaremos o slice:
print("Lista das palavras semanticamente MAIS DISTANTES de \'título\': ", lista_similaridade[0:3]) 

Lista das palavras semanticamente MAIS PRÓXIMAS de 'título':  [(0.41359764, 'vitória'), (0.4326085, 'terceiro'), (0.59572226, 'troféu'), (1.0, 'título')]
Lista das palavras semanticamente MAIS DISTANTES de 'título':  [(-0.2338472, 'e'), (-0.17984475, 'com'), (-0.16082522, 'ficam')]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.


2. Encontrar os vetores de todas as palavras do córpus e descobrir quais são as palavras relacionadas.

In [ ]:
# Esse exercício é parecido com o anterior, só que dessa vez iremos comparar TODOS OS TOKENS com TODOS OS TOKENS!

# Linhas de código padrão! Já estamos bem nisso!
nlp = spacy.load("pt_core_news_lg")
doc = nlp(conteudo)
tokens = [token for token in doc]

lista_similaridade = []

# Vamos precisar de mais poder aqui. Iremos usar uma estrutura de repetição dentro de outra, para passar por todos os tokens:
for token in tokens[:10]:

  # Identificando a palavra1 da nossa tupla (palavra1, palavra2)
  palavra1 = token

  for token_seguinte in tokens[:10]:
    if palavra1 != token_seguinte:
      palavra2 = token_seguinte

      similaridade = palavra1.similarity(palavra2)
      info = (similaridade, palavra1.orth_, palavra2.orth_)

      if info not in lista_similaridade:
        lista_similaridade.append(info)

lista_similaridade.sort()
quantidade = 5

print("As", quantidade, "palavras mais relacionadas por sua similaridade são: ")
print(lista_similaridade[-quantidade::])




As 5 palavras mais relacionadas por sua similaridade são: 
[(0.44188765, 'Patriots', 'Bowl'), (0.5228125, 'Bowl', 'Giants'), (0.5228125, 'Giants', 'Bowl'), (0.6893943, 'Giants', 'Patriots'), (0.6893943, 'Patriots', 'Giants')]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
